In [1]:
import pandas as pd

pd.set_option('display.max_colwidth', 200)
df = pd.read_csv("../result/gene_classification_gnn_hp_tune.csv")

In [2]:
default_lr = "1e-02"
default_dropout = "0.1"
default_weight_decay = "1e-05"

optim_gcn_dim = "64"
optim_gcn_num_layers = "5"
optim_gcn_residual = "1"

optim_sage_dim = "64"
optim_sage_num_layers = "5"
optim_sage_residual = "0"

In [3]:
def print_best(
    name,
    df,
    sep_width=100,
    top_k=5,
    sortby="Validation score",
):  
    df_top = (
        df
        .sort_values("Validation score", ascending=False)
        .head(top_k)
        .drop("RunID", axis=1)
        .reset_index(drop=True)
    )
    optim_settings = df_top.iloc[0]["Settings"]
    
    name_str = "_".join(name) if isinstance(name, tuple) else name
    print(name_str)
    print("=" * sep_width)
    print(f"Optimal settings: {optim_settings}")
    print(df_top)
    print("-" * sep_width)
    print()
    
    
def summarize(df):
    return (
        df
        .groupby(["Network", "Method", "Settings", "Task"], as_index=False)
        .median()  # Take the median across different runs
        .groupby(["Method", "Settings"], as_index=False)
        .mean()  # Take the average across all networks and datsets
    )

## GNN architecture tuning

In [4]:
# GNN architecture tuning results (fixed default training params)
default_settings = f"_lr={default_lr}_dropout={default_dropout}_weight-decay={default_weight_decay}"

df_architecture = df[df["Settings"].str.endswith(default_settings)].reset_index(drop=True).copy()
df_architecture["Settings"] = df_architecture["Settings"].str.replace(default_settings, "", regex=False)
df_architecture

,Training score,Validation score,Testing score,Task,Dataset,Network,Method,Settings,RunID
0,0.577118,0.598003,0.246031,GO:0006650,GOBP,HumanBaseTop-global,sage,dim=16_num-layers=5_residual=0,2
1,2.711785,1.671210,0.605706,GO:0006402,GOBP,HumanBaseTop-global,sage,dim=16_num-layers=5_residual=0,2
2,0.566202,0.108797,0.735409,GO:0010498,GOBP,HumanBaseTop-global,sage,dim=16_num-layers=5_residual=0,2
3,0.112633,0.118376,0.041880,GO:0006979,GOBP,HumanBaseTop-global,sage,dim=16_num-layers=5_residual=0,2
4,0.639710,-0.078356,1.664527,GO:0097190,GOBP,HumanBaseTop-global,sage,dim=16_num-layers=5_residual=0,2
...,...,...,...,...,...,...,...,...,...
121550,0.814313,0.543941,0.008472,GO:0007033,GOBP,HumanBaseTop-global,gcn,dim=16_num-layers=2_residual=1,4
121551,1.917618,0.174727,1.139742,GO:0008033,GOBP,HumanBaseTop-global,gcn,dim=16_num-layers=2_residual=1,4
121552,0.651959,0.942165,0.655576,GO:0030148,GOBP,HumanBaseTop-global,gcn,dim=16_num-layers=2_residual=1,4
121553,0.470179,0.594316,-0.234940,GO:0006865,GOBP,HumanBaseTop-global,gcn,dim=16_num-layers=2_residual=1,4


In [5]:
# Optimal GNN architectures across all networks and tasks
for method, g in summarize(df_architecture).groupby("Method"):
    print_best(method, g.drop(["Method", "Training score"], axis=1))

gcn
Optimal settings: dim=64_num-layers=5_residual=1
                          Settings  Validation score  Testing score
0   dim=64_num-layers=5_residual=1          0.863414       0.875520
1  dim=128_num-layers=5_residual=1          0.841311       0.822184
2   dim=32_num-layers=5_residual=1          0.812789       0.785182
3   dim=64_num-layers=4_residual=1          0.805117       0.799584
4  dim=128_num-layers=4_residual=1          0.801518       0.791148
----------------------------------------------------------------------------------------------------

sage
Optimal settings: dim=64_num-layers=5_residual=0
                          Settings  Validation score  Testing score
0   dim=64_num-layers=5_residual=0          0.876487       0.790634
1   dim=32_num-layers=5_residual=0          0.876236       0.838969
2   dim=32_num-layers=4_residual=0          0.800093       0.749191
3   dim=64_num-layers=4_residual=0          0.797512       0.780895
4  dim=128_num-layers=5_residual=0         

In [6]:
# Optimal GNN architecture specific to netowork and task
if False:  # switch to True to enable
    grouped = (
        df_architecture
        .groupby(["Network", "Method", "Dataset", "Task", "Settings"], as_index=False)
        .median()  # Take the median across different runs
        .groupby(["Network", "Method", "Dataset", "Settings"], as_index=False)
        .mean()  # Take the average across all networks and datsets
        .groupby(["Method", "Network", "Dataset"])
    )

    for method, g in grouped:
        print_best(
            method,
            g.drop(["Method", "Network", "Dataset", "Training score"], axis=1),
        )

## Training parameters tuning

In [7]:
# GCN training parameter tuning results (fixed optimal architecture)
gcn_default_settings = f"dim={optim_gcn_dim}_num-layers={optim_gcn_num_layers}_residual={optim_gcn_residual}_"
sage_default_settings = f"dim={optim_sage_dim}_num-layers={optim_sage_num_layers}_residual={optim_sage_residual}_"

df_params_gcn = df[(df["Method"] == "gcn") & df["Settings"].str.startswith(gcn_default_settings)]
df_params_sage = df[(df["Method"] == "sage") & df["Settings"].str.startswith(sage_default_settings)]

df_params = pd.concat((df_params_gcn, df_params_sage)).reset_index(drop=True).copy()
df_params["Settings"] = (
    df_params["Settings"]
    .str.replace(gcn_default_settings, "", regex=False)
    .str.replace(sage_default_settings, "", regex=False)
)
df_params

,Training score,Validation score,Testing score,Task,Dataset,Network,Method,Settings,RunID
0,0.616518,0.455194,-0.124688,GO:0006650,GOBP,HumanBaseTop-global,gcn,lr=1e-02_dropout=0.0_weight-decay=1e-04,2
1,3.101985,1.429446,0.844169,GO:0006402,GOBP,HumanBaseTop-global,gcn,lr=1e-02_dropout=0.0_weight-decay=1e-04,2
2,0.691949,0.177387,0.536794,GO:0010498,GOBP,HumanBaseTop-global,gcn,lr=1e-02_dropout=0.0_weight-decay=1e-04,2
3,0.977094,0.367755,0.307089,GO:0006979,GOBP,HumanBaseTop-global,gcn,lr=1e-02_dropout=0.0_weight-decay=1e-04,2
4,0.767894,0.967999,0.898800,GO:0097190,GOBP,HumanBaseTop-global,gcn,lr=1e-02_dropout=0.0_weight-decay=1e-04,2
...,...,...,...,...,...,...,...,...,...
209275,0.000000,0.000000,0.000000,GO:0007033,GOBP,HumanBaseTop-global,sage,lr=1e-01_dropout=0.0_weight-decay=1e-07,4
209276,0.003504,0.002102,0.000000,GO:0008033,GOBP,HumanBaseTop-global,sage,lr=1e-01_dropout=0.0_weight-decay=1e-07,4
209277,0.000000,0.000000,0.000000,GO:0030148,GOBP,HumanBaseTop-global,sage,lr=1e-01_dropout=0.0_weight-decay=1e-07,4
209278,0.000000,0.000000,0.000000,GO:0006865,GOBP,HumanBaseTop-global,sage,lr=1e-01_dropout=0.0_weight-decay=1e-07,4


In [8]:
# Optimal training parameters across all networks and tasks
for method, g in summarize(df_params).groupby("Method"):
    print_best(method, g.drop(["Method", "Training score"], axis=1))

gcn
Optimal settings: lr=1e-02_dropout=0.1_weight-decay=1e-06
                                  Settings  Validation score  Testing score
0  lr=1e-02_dropout=0.1_weight-decay=1e-06          0.876762       0.869555
1  lr=1e-02_dropout=0.1_weight-decay=1e-05          0.863414       0.875520
2  lr=1e-02_dropout=0.1_weight-decay=1e-07          0.854135       0.819888
3  lr=1e-02_dropout=0.3_weight-decay=1e-06          0.826009       0.752661
4  lr=1e-02_dropout=0.3_weight-decay=1e-07          0.824807       0.811127
----------------------------------------------------------------------------------------------------

sage
Optimal settings: lr=1e-03_dropout=0.1_weight-decay=1e-05
                                  Settings  Validation score  Testing score
0  lr=1e-03_dropout=0.1_weight-decay=1e-05          1.021336       0.940476
1  lr=1e-03_dropout=0.1_weight-decay=1e-07          1.009357       0.934350
2  lr=1e-03_dropout=0.1_weight-decay=1e-06          1.001497       0.950301
3  lr=1e-03_d

In [9]:
# Optimal training parameters specific to netowork and task
if False:  # switch to True to enable
    grouped = (
        df_params
        .groupby(["Network", "Method", "Dataset", "Task", "Settings"], as_index=False)
        .median()  # Take the median across different runs
        .groupby(["Network", "Method", "Dataset", "Settings"], as_index=False)
        .mean()  # Take the average across all networks and datsets
        .groupby(["Method", "Network", "Dataset"])
    )

    for method, g in grouped:
        print_best(
            method,
            g.drop(["Method", "Network", "Dataset", "Training score"], axis=1),
        )